In [0]:
%run "./15 Intro to Delta Tables"

Uso de JOINS

In [0]:
-- Creamos una tabla para mostrar los joins

CREATE OR REPLACE TABLE addresses_demo (
  id    INT,
  city  STRING,
  state STRING
)
USING DELTA;

INSERT INTO addresses_demo
VALUES
  (1, 'San José',    'San José'),
  (2, 'Alajuela',    'Alajuela'),
  (3, 'Heredia',     'Heredia'),
  (11, 'San José',    'San José'),
  (12, 'Alajuela',    'Alajuela'),
  (13, 'Heredia',     'Heredia'),
  (21, 'San José',    'San José'),
  (22, 'Alajuela',    'Alajuela'),
  (23, 'Heredia',     'Heredia'),
  (31, 'San José',    'San José'),
  (32, 'Alajuela',    'Alajuela'),
  (33, 'Heredia',     'Heredia'),
  (41, 'San José',    'San José'),
  (42, 'Alajuela',    'Alajuela'),
  (43, 'Heredia',     'Heredia'),
  (51, 'San José',    'San José'),
  (52, 'Alajuela',    'Alajuela'),
  (53, 'Heredia',     'Heredia'),
  (61, 'San José',    'San José'),
  (62, 'Alajuela',    'Alajuela'),
  (63, 'Heredia',     'Heredia'),
  (71, 'San José',    'San José'),
  (72, 'Alajuela',    'Alajuela'),
  (73, 'Heredia',     'Heredia'),
  (81, 'San José',    'San José'),
  (82, 'Alajuela',    'Alajuela'),
  (83, 'Heredia',     'Heredia'),
  (91, 'San José',    'San José'),
  (92, 'Alajuela',    'Alajuela'),
  (93, 'Heredia',     'Heredia'),
  (106, 'Cartago',     'Cartago'),   -- id 106 no existe 
  (116, 'Cartago',     'Cartago'),   -- id 116 no existe 
  (126, 'Cartago',     'Cartago');   -- id 126 no existe 

In [0]:
DESCRIBE EXTENDED addresses_demo

# INNER JOIN

In [0]:
SELECT
  p.id,
  p.name,
  p.age,
  a.city,
  a.state
FROM people_delta p
INNER JOIN addresses_demo a
  ON p.id = a.id;

# FULL OUTER JOIN

NOTA: La función COALESCE devuelve el primer valor no nulo de la lista de expresiones que recibe

In [0]:
SELECT
  COALESCE(p.id, a.id) AS id,
  p.name,
  p.age,
  a.city,
  a.state
FROM people_delta p
FULL OUTER JOIN addresses_demo a
  ON p.id = a.id
ORDER BY id;

# UNION
UNION (quita duplicados)


In [0]:
-- Subset A: menores de 35
CREATE OR REPLACE TEMP VIEW under_35 AS
SELECT id, name, age FROM people_delta
WHERE age < 35;

-- Subset B: mayores de 60
CREATE OR REPLACE TEMP VIEW over_60 AS
SELECT id, name, age FROM people_delta
WHERE age > 60;

-- Union de los dos conjuntos (sin duplicados)
SELECT * FROM under_35
UNION
SELECT * FROM over_60;

# INTERSECT

Sólo filas comunes entre dos conjuntos

In [0]:
SELECT * FROM under_35
INTERSECT
SELECT * FROM over_60;

In [0]:
SELECT * FROM under_35
INTERSECT
SELECT * FROM people_delta;

# EXCEPT
Filas en A que no están en B

In [0]:
SELECT * FROM over_60
EXCEPT
SELECT * FROM (
  SELECT id, name, age FROM people_delta WHERE age > 65
);

# PIVOT
Cambiar perspectiva: contar personas por grupo de edad en cada ciudad


In [0]:
CREATE OR REPLACE TEMP VIEW person_addr AS
SELECT
  p.id,
  p.name,
  p.age,
  a.city,
  CASE
    WHEN p.age < 35 THEN 'Adulto Joven'
    WHEN p.age < 65 THEN 'Adulto'
    ELSE 'Adulto Mayor'
  END AS age_group
FROM people_delta p
LEFT JOIN addresses_demo a
  ON p.id = a.id;

In [0]:
SELECT * FROM person_addr
ORDER BY city

Pivot: contar por age_group en cada city

In [0]:
SELECT * FROM (
  SELECT city, age_group FROM person_addr
)
PIVOT (
  COUNT(*) FOR age_group IN ('Adulto Joven','Adulto' ,  'Adulto Mayor')
)
ORDER BY city;